In [10]:
import api

In [11]:
def materia(nombre: str):
    l = api.lista_materias('materias.json') # datos precargados
    m = api.encontrar_materia(l, nombre)[0][0]
    # return api.datos_materia(m)
    return m

In [36]:
from api import *

horario = api.horario_curso(3, 1, 3, 'materias.json')
horario = api.horario_materia(horario, materia("Objetos"), 2)
print(horario.fillna(''))

[Conflicto] POO E1 / IA I3 - Martes 15:30
         Lunes          Martes Miércoles   Jueves Viernes
09:00    IA E1           IS E1     IA E1   ASR E1  ASR I3
09:30    IA E1           IS E1     IA E1   ASR E1  ASR I3
10:00    CD E1         TALF E1     IA E1  TALF E1  ASR I3
10:30    CD E1         TALF E1     IA E1  TALF E1  ASR I3
11:00    IS E1                     CD E1  TALF E1        
11:30    IS E1                     CD E1  TALF E1   IS I3
12:00  TALF I3          ASR I3    POO I2            IS I3
12:30  TALF I3          ASR I3    POO I2            IS I3
13:00  TALF I3          ASR I3    POO I2            IS I3
13:30  TALF I3          ASR I3    POO I2            IS I3
14:00                             POO I2                 
14:30                                                    
15:00                   POO E1                           
15:30           IA I3 / POO E1                      CD I3
16:00           IA I3 / POO E1                      CD I3
16:30           IA I3 / POO E1